In [2]:
import numpy as np
import pandas as pd
import random
# from functools import reduce
from scipy.ndimage import generic_filter


class mineSweeper:
    def __init__(self, position=[2,2], map_size=8, mines_ratio=1):
        self.size = map_size
        self.mines = round((mines_ratio/10)*(map_size**2))
        self.generated_map = pd.DataFrame(np.full((self.size,self.size),0))
        # self.counted_map = pd.DataFrame(np.full((self.size,self.size),0)) ##optimizing memory
        self.revealed_map = pd.DataFrame(np.full((self.size,self.size),False))
        self.flagged_map = pd.DataFrame(np.full((self.size,self.size),False)) ##optimizing memory failed
        self.win_flag = False
        self.terminate_flag = False
        self.start_position = (((int(position[0]))*self.size)+((int(position[1])))) if len(position) else self.size**2/2

         #incorporating first safe position


    def generate_map(self):
        random_numbers = random.sample(range(0,self.size**2),self.mines)
        for i in random_numbers:
            [col,row] = [i//self.size,i%self.size]
            self.generated_map.iat[col,row]="M"
        self.generated_map = self.count_map(self.generated_map)

    def check_window(self,values):
        surround_count = int(np.count_nonzero(values)) if values[4]!=9 else 9
        return surround_count

    def count_map(self,check_map):
        counted_map = generic_filter(check_map, self.check_window, size=(3,3), mode='constant', cval=0)
        # for i in counted_map:
        #     for j in i:
        #         if j==9:
        #             i="M"
        return pd.DataFrame(counted_map)

class userAction(mineSweeper):

    def generate_map(self): ## ensuring first forgiveness (may increase forgiveness values)
        self.revealed_map = pd.DataFrame(np.full((self.size,self.size),False))
        self.flagged_map = pd.DataFrame(np.full((self.size,self.size),False)) ##optimizing memory failed
        self.win_flag = False
        self.terminate_flag = False
        i = self.start_position//self.size
        j = self.start_position%self.size
        safe_zone =[[x,y] for x in
                        range(max(i-1, 0),min(i+2,self.size)) for y in
                        range(max(j-1, 0),min(j+2,self.size))]

        random_numbers = random.sample( [x for x in range(0,self.size**2)
                                         if x != self.start_position],self.mines)
        for i in random_numbers:
            [col,row] = [i//self.size,i%self.size]
            self.generated_map.iat[col,row]=9
        self.generated_map = self.count_map(self.generated_map)
        self.reveal_map([self.start_position//self.size,self.start_position%self.size])


    def visible_map(self):
        visible_map = self.generated_map.where(self.revealed_map)
        for i in range(self.size):
            for j in range(self.size):
                if self.flagged_map.iat[i,j]:
                    visible_map.iat[i,j] = "F"
        return visible_map

    def reveal_map(self,position):
    #     [i,j] = position
        i,j = int(position[0]),int(position[1])
        # print ("positions", self.generated_map)
        if self.revealed_map.iat[i,j] != "F":

            self.revealed_map.iat[i,j] = True
            # self.revealed_map.iat[i,j] = True
            self.win_status()
            if self.generated_map.iat[i,j] in ["M",9]:
                print("Mines Triggered - Objective Failed",self.visible_map())
                self.terminate_flag = True
                return -1.0 ##scores for training
            elif self.generated_map.iat[i,j] == 0 and not self.revealed_map.iat[i,j]:
                # self.revealed_map.iat[i,j] = True
                surrounding_positions = [[x,y] for x in
                        range(max(i-1, 0),min(i+2,self.size)) for y in
                        range(max(j-1, 0),min(j+2,self.size))]
                print("Surroundings safe, unlocking more area",surrounding_positions)
                for i in surrounding_positions:
                    self.reveal_map(i)
                #reduce(self.reveal_map(x),surrounding_positions)
                return 0.05*len(surrounding_positions)
            else:
                print("Location revealed",i,j)
                return 0.01
        else:
            print("Location flagged")
            return -0.1

    def flag_map(self,position):
        [i,j] = position
        print("Flagged location",i,j)
        if not self.flagged_map.iat[i,j]:
            print("Already flagged")
            return -0.1
        else:
            if (self.flagged_map==True).sum().sum()<self.mines:
                self.flagged_map.iat[i,j] = True
                print("Flag consumed")
                return 0.1
            elif (self.flagged_map==True).sum().sum()<self.mines:
                print("Insufficient flags")
                return -0.1
            else:
                return self.win_status()

    def unflag_map(self,position):
        [i,j] = position
        if self.flagged_map.iat[i,j]:
            print("Unflagged")
            self.flagged_map.iat[i,j] = False
        else:
            print("No flag placed")

    def win_status(self):
        if (self.revealed_map).sum().sum()+self.mines == self.size**2:
            print("All mines marked")
            self.win_flag = True
            self.terminate_flag=True
            return 1.0
        else:
            print("Insufficient map revealed")
            return 0.0

    ## traininic specifics required
    def to_numpy_tensor(self):
        revealed = self.revealed_map.to_numpy().astype(np.float32)
        flagged = self.flagged_map.to_numpy().astype(np.float32)
        board = self.generated_map.replace("M", 9).fillna(0).to_numpy().astype(np.float32)
        board_masked = board * revealed  # only show revealed numbers
        return np.stack([board_masked, revealed, flagged], axis=0)  # shape: (3, size, size)

    def training_action(self,action): ##wrapper to enable easier training
        if action =="start":
            return(self.generate_map(),self.to_numpy_tensor(),self.terminate_flag,self.win_status())
        a,x,y = action.split(",")
        x= int(x)
        y= int(y)
        if a == "f":
            return(self.flag_map([x,y]),self.to_numpy_tensor(),self.terminate_flag,self.win_status(),self.revealed_map)
        elif a == "u":
            return(self.unflag_map([x,y]),self.to_numpy_tensor(),self.terminate_flag,self.win_status(),self.revealed_map)
        elif a == 'r':
            return(self.reveal_map([x,y]),self.to_numpy_tensor(),self.terminate_flag,self.win_status(),self.revealed_map)
        else:
            pass

    def llm_training_action(self,action):
        while not self.terminate_flag:
            response_provision = (self.stringify_visible_map,"Please choose next action as \nf:for flag, \nu:for unflag, \nr:for reveal followed by x,y coordinates\n i.e. as action,x-coordinate,y-coordinate")
            try:
                action,x,y = response_provision.split(",")
                x= int(x)-1
                y= int(y)-1
                if action == "f":
                    return(self.flag_map([x,y]),self.terminate_flag,self.win_status(),response_provision)
                elif action == "u":
                    return(self.unflag_map([x,y]),self.terminate_flag,self.win_status(),response_provision)
                else:
                    return(self.unflag_map([x,y]),self.terminate_flag,self.win_status(),response_provision)
            except:
                print("Please enter valid input")
            return response_provision
        if execute_game_launcher.win_status() is True:
            pass
        restart = True if input(print("Would you like to restart? y/n"))not in ['n','no', 'end-game'] else False

    def stringify_visible_map(self):
        visible = self.visible_map().to_numpy()
        return "\\n".join([" ".join(str(cell) if pd.notna(cell) else "#" for cell in row) for row in visible])


def run_mcts(action:userAction):
    # valid_actions = valid_actions*action.count_map(action.visible_map())
    action_probs = action.count_map(action.visible_map().fillna(1))
    valid_actions = [
        action_probs.iat[i,j] if not action.revealed_map.iat[i, j] and not action.flagged_map.iat[i, j] and action_probs.iat[i,j] != 9 else 0.2*(not action.revealed_map.iat[i, j])
        for i in range(action.size) for j in range(action.size)]

    print("Action probables",action_probs,"before filter",action.revealed_map,"invisible",action.generated_map)

    valid_sum = sum([x for x in valid_actions])+1

    if valid_sum==1: ## debug sequence for analysis
        action.terminate_flag = True
        action.win_status()
        return None
        # print("Action probables",action_probs,"before filter",action.revealed_map,"invisible",action.generated_map)
        # if input(print("Continue?")): pass

    print("Action bug",valid_actions,valid_sum)
    valid_actions = [x / valid_sum for x in valid_actions]
    # print("Action probables",valid_actions)
    # action_probs = torch.tensor(valid_actions)

    # print("Action probables",action_probs,"converted")
    value = valid_sum
    return action_probs


In [ ]:
from peft import LoraConfig, TaskType, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np

# from alphaZero import run_mcts
# from minesweeperGenerator import userAction


def load_tiny_llama_lora():
    base_model = AutoModelForCausalLM.from_pretrained("TinyLLaMA/TinyLLaMA-1.1B-Chat", device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained("TinyLLaMA/TinyLLaMA-1.1B-Chat")

    lora_config = LoraConfig(
        r=8,
        lora_alpha=16,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=0.1,
        bias="none",
        task_type=TaskType.CAUSAL_LM
    )

    model = get_peft_model(base_model, lora_config)
    return model, tokenizer

llama_model, llama_tokenizer = load_tiny_llama_lora()

def llama_agent_step(obs):
    prompt = f"You are playing Minesweeper. Board state:\n{obs}\nWhich cell should be uncovered next (return the index)?"
    inputs = llama_tokenizer(prompt, return_tensors="pt").to(llama_model.device)
    outputs = llama_model.generate(**inputs, max_new_tokens=10)
    response = llama_tokenizer.decode(outputs[0], skip_special_tokens=True)
    try:
        action = int([s for s in response.split() if s.isdigit()][-1])
    except:
        action = np.random.randint(run_mcts(userAction))
    return action

game = userAction()
game.training_action("start")
done = False
while not done:
    # obs = obs.stringify_visible_map()
    action = llama_agent_step(obs)
    reward, done, win, obs = game.llm_training_action(action)

print("TinyLLaMA agent finished with reward:", reward+win*10 )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: TinyLLaMA/TinyLLaMA-1.1B-Chat is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
# Run TinyLLaMA inference in Google Colab without authentication
!pip install llama-cpp-python
!wget https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-GGUF/resolve/main/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf -O tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf

from llama_cpp import Llama
import numpy as np
# from alphaZero import run_mcts
# from minesweeperGenerator import userAction

# Load TinyLLaMA GGUF model using llama.cpp
llama_model = Llama(
    model_path="tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf",  # Downloaded file
    n_ctx=2048,
    n_threads=4,
    use_mlock=False  # Disabled in Colab
)

def llama_agent_step(obs):
    prompt = f"You are playing Minesweeper. Board state:\n{obs}\nWhich cell should be uncovered next (return the index)?"
    output = llama_model(
        prompt,
        max_tokens=16,
        stop=["\n"],
        echo=False
    )
    response = output["choices"][0]["text"]
    try:
        action = int([s for s in response.split() if s.isdigit()][-1])
    except:
        action = np.random.randint(run_mcts(userAction))
    return action

# Initialize the Minesweeper environment
game = userAction()
game.training_action("start")
done = False

# Main gameplay loop
while not done:
    visible_map = game.stringify_visible_map()
    action = llama_agent_step(visible_map)
    reward, done, win, visible_map = game.llm_training_action(action)

print("TinyLLaMA agent finished with reward:", reward + win * 10)


  Using cached llama_cpp_python-0.3.12.tar.gz (49.8 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.6 MB/s eta 0:00:00
